In [1]:
#hide
#default_exp tools.change_manager
from nbdev.showdoc import *
from dsblocks.utils.nbdev_utils import nbdev_setup, TestRunner

nbdev_setup ()
tst = TestRunner (targets=['dummy'])

# Change manager

Displays the relationship between scores obtained across experiments with two different evaluation metrics indicated as argument. Calculates Pearson R-squared coefficient.

In [2]:
#export
import sys
import argparse
from importlib import reload
import warnings
warnings.filterwarnings('ignore')

import pdb

import hpsearch.config.hp_defaults as dflt

In [3]:
#for tests
import os
import pytest
from pathlib import Path
import cloudpickle

from dsblocks.utils.utils import remove_previous_results

from hpsearch.config.manager_factory import ManagerFactory
from hpsearch.examples.example_experiment_manager import ExampleExperimentManager
from hpsearch.examples.dummy_experiment_manager import DummyExperimentManager

## change_manager

In [4]:
#export
def change_manager (subclass=None, manager_path=dflt.manager_path, print_list=True):
    # TODO: check if it is really necessary to import inside the function and reload
    import hpsearch.config.manager_factory as mf
    from hpsearch.config.hpconfig import register_manager
    reload (mf)

    manager_factory = mf.ManagerFactory (manager_path=manager_path, verbose=2)
    
    if subclass is None:
        print ('\nList of managers:')
        manager_factory.list_subclasses ()
    else:
        manager_factory.change_manager (subclass)
        em = manager_factory.get_experiment_manager ()
        register_manager (em, manager_path=manager_factory.manager_path)
        if print_list:
            change_manager (subclass=None, manager_path=manager_path)

### example usage

In [5]:
#export tests.tools.test_change_manager
def test_change_manager_list ():
    # the following lists stored experiment managers, keeping the current experiment manager
    path_results = 'test_change_manager_list'
    manager_path = f'{path_results}/manager'
    path_experiments = f'{path_results}/experiments'
    
    manager_factory = ManagerFactory (manager_path=manager_path)
    manager_factory.delete_and_reset_all ()

    assert os.path.exists (path_results)
    assert os.listdir (path_results)==[]

    print ('\ninitial list')
    em_before = manager_factory.get_experiment_manager ()
    change_manager (None, manager_path=manager_path)
    em_after = manager_factory.get_experiment_manager ()
    assert em_before.__class__.__name__ == em_after.__class__.__name__

    assert os.listdir (manager_path)==['logs.txt']

    # **************************************************************
    # same but using ExampleExperimentManager as registered manager
    # **************************************************************
    em = ExampleExperimentManager(path_experiments=path_experiments)
    manager_factory.register_manager (em)
    manager_factory.write_manager (em)

    assert sorted(os.listdir (manager_path))==['fields', 'info', 'logs.txt', 'whole']
    assert sorted(os.listdir (f'{manager_path}/whole'))==['ExampleExperimentManager-experiments.pk', 'last.pk']
    assert sorted(os.listdir (f'{manager_path}/info'))==['ExampleExperimentManager-experiments.pk', 'last.pk']
    assert sorted(os.listdir (f'{manager_path}/fields'))==['ExampleExperimentManager-experiments.pk', 'last.pk']

    print ('\nlist after storing ExampleExperimentManager')
    em_before = manager_factory.get_experiment_manager ()
    change_manager (None, manager_path=manager_path)
    em_after = manager_factory.get_experiment_manager ()
    assert em_before.__class__.__name__ == em_after.__class__.__name__

    assert sorted(os.listdir (f'{manager_path}/whole'))==['ExampleExperimentManager-experiments.pk', 'last.pk']

    # we store a third EM (DummyExperimentManager) and list the two 
    # stored managers
    print ('\nlist after storing DummyExperimentManager')
    em = DummyExperimentManager (path_experiments=path_experiments)
    manager_factory.register_manager(em)
    manager_factory.write_manager(em)
    assert manager_factory.get_experiment_manager().__class__.__name__ == 'DummyExperimentManager'
    change_manager (None, manager_path=manager_path)

    assert sorted(os.listdir (f'{manager_path}/whole'))==['DummyExperimentManager-experiments.pk', 'ExampleExperimentManager-experiments.pk', 'last.pk']
    
    whole_object_path = Path(f'{manager_path}/whole')
    em=cloudpickle.load (open (whole_object_path / 'last.pk', 'rb'))
    assert em.__class__.__name__=='DummyExperimentManager'
    
    # we change the registered manager back to the first one
    print ('\nlist after changing manager')
    change_manager('ExampleExperimentManager-experiments', manager_path=manager_path)
    assert manager_factory.get_experiment_manager().__class__.__name__ == 'ExampleExperimentManager'
    em=cloudpickle.load (open (whole_object_path / 'last.pk', 'rb'))
    assert em.__class__.__name__=='ExampleExperimentManager'
    
    remove_previous_results (path_results)

In [6]:
tst.run (test_change_manager_list, tag='dummy', debug=False)

experiment manager not registered yet, importing experiment manager
loading manager from /home/jcidatascience/jaume/workspace/remote/temp/hpsearch/test_change_manager_list/manager/whole
No experiment manager to import was found, setting base manager.
experiment manager not registered yet, importing experiment manager
loading manager from /home/jcidatascience/jaume/workspace/remote/temp/hpsearch/test_change_manager_list/manager/whole
returning registered experiment manager
returning experiment manager <hpsearch.examples.example_experiment_manager.ExampleExperimentManager object at 0x7fb633d00460>
returning experiment manager <hpsearch.examples.example_experiment_manager.ExampleExperimentManager object at 0x7fb633d00460>
returning registered experiment manager
returning experiment manager <hpsearch.examples.example_experiment_manager.ExampleExperimentManager object at 0x7fb633d00460>
experiment manager not registered yet, importing experiment manager
loading manager from /home/jcidatasci

running test_change_manager_list

initial list
managers: []
experiment manager registered: ExperimentManager
registered name: ExperimentManager-default

list after storing ExampleExperimentManager
managers: ['ExampleExperimentManager-default']
experiment manager registered: ExampleExperimentManager
registered name: ExampleExperimentManager-default

list after storing DummyExperimentManager
managers: ['DummyExperimentManager-default', 'ExampleExperimentManager-default']
experiment manager registered: DummyExperimentManager
registered name: DummyExperimentManager-default

list after changing manager
managers: ['DummyExperimentManager-default', 'ExampleExperimentManager-default']
experiment manager registered: ExampleExperimentManager
registered name: ExampleExperimentManager-default


## parse_args

In [7]:
#export
def parse_args (args):
    parser = argparse.ArgumentParser(description='change experiment manager')
    parser.add_argument('-m','--manager', type=str, default=None, help="new experiment manager to use")
    parser.add_argument('-l', '--list', action= "store_true", help="list experiment managers registered so far")
    parser.add_argument('-p','--path', type=str, default=None, 
                        help=f"path where experiment managers are stored, default={dflt.manager_path}")
    pars = parser.parse_args(args)
    return pars

## run_change_manager

In [8]:
#export
def run_change_manager (pars):
    manager_path = dflt.manager_path if pars.path is None else pars.path
    #pdb.set_trace()
    if pars.list:
        change_manager (subclass=None, manager_path=manager_path)
    else:
        if pars.manager is None:
            print ('you did not indicate any manager to change to, listing the managers available')
        change_manager (subclass=pars.manager, manager_path=manager_path)

## parse_arguments_and_run

In [9]:
#export
def parse_arguments_and_run (args):
    pars = parse_args(args)
    run_change_manager (pars)

### Example usage

In [16]:
#export tests.tools.test_change_manager
def test_change_manager_parse_and_run ():
    # the following lists stored experiment managers, keeping the current experiment manager
    path_results = 'test_change_manager_parse_and_run'
    manager_path = f'{path_results}/manager'
    path_experiments = f'{path_results}/experiments'
    
    # *********************************
    # write managers
    # *********************************
    em = ExampleExperimentManager(path_experiments=path_experiments, manager_path=manager_path)
    em.register_and_store_subclassed_manager ()
    
    em = DummyExperimentManager(path_experiments=path_experiments, manager_path=manager_path)
    em.register_and_store_subclassed_manager ()
    
    assert sorted(os.listdir (f'{manager_path}/whole'))==['DummyExperimentManager-experiments.pk', 'ExampleExperimentManager-experiments.pk', 'last.pk']
    
    # *********************************
    # *********************************
    command = f'-l -p {manager_path}'
    parse_arguments_and_run (command.split())
    
    command = f'-m ExampleExperimentManager-experiments -p {manager_path}'
    parse_arguments_and_run (command.split())
    
    command = f'-l -p {manager_path}'
    parse_arguments_and_run (command.split())
    
    remove_previous_results (path_results)

In [17]:
tst.run (test_change_manager_parse_and_run, tag='dummy', debug=False)

experiment manager not registered yet, importing experiment manager
loading manager from /home/jcidatascience/jaume/workspace/remote/temp/hpsearch/test_change_manager_parse_and_run/manager/whole
returning registered experiment manager
returning experiment manager <hpsearch.examples.dummy_experiment_manager.DummyExperimentManager object at 0x7fb6581b8490>
returning experiment manager <hpsearch.examples.dummy_experiment_manager.DummyExperimentManager object at 0x7fb6581b8490>
experiment manager not registered yet, importing experiment manager
loading manager from /home/jcidatascience/jaume/workspace/remote/temp/hpsearch/test_change_manager_parse_and_run/manager/whole
returning registered experiment manager
returning experiment manager <hpsearch.examples.dummy_experiment_manager.DummyExperimentManager object at 0x7fb602008df0>
returning experiment manager <hpsearch.examples.dummy_experiment_manager.DummyExperimentManager object at 0x7fb602008df0>
loading manager from /home/jcidatascience/

running test_change_manager_parse_and_run
managers: ['DummyExperimentManager-default', 'ExampleExperimentManager-default']
experiment manager registered: DummyExperimentManager
registered name: DummyExperimentManager-default
managers: ['DummyExperimentManager-default', 'ExampleExperimentManager-default']
experiment manager registered: ExampleExperimentManager
registered name: ExampleExperimentManager-default
managers: ['DummyExperimentManager-default', 'ExampleExperimentManager-default']
experiment manager registered: ExampleExperimentManager
registered name: ExampleExperimentManager-default


## main

In [ ]:
#export
def main():
    parse_arguments_and_run (sys.argv[1:])